In [1]:
import duckdb

In [3]:
con = duckdb.connect('data/exploitation.db')

In [ ]:
con.execute("show tables").fetchall()

[('dataset_test',),
 ('dataset_train',),
 ('houses',),
 ('mappings',),
 ('mappings_subset',),
 ('sandbox_0',),
 ('sandbox_1',),
 ('sandbox_10',),
 ('sandbox_11',),
 ('sandbox_12',),
 ('sandbox_13',),
 ('sandbox_14',),
 ('sandbox_15',),
 ('sandbox_16',),
 ('sandbox_17',),
 ('sandbox_18',),
 ('sandbox_19',),
 ('sandbox_2',),
 ('sandbox_20',),
 ('sandbox_21',),
 ('sandbox_22',),
 ('sandbox_23',),
 ('sandbox_24',),
 ('sandbox_25',),
 ('sandbox_26',),
 ('sandbox_27',),
 ('sandbox_28',),
 ('sandbox_29',),
 ('sandbox_3',),
 ('sandbox_30',),
 ('sandbox_31',),
 ('sandbox_32',),
 ('sandbox_33',),
 ('sandbox_34',),
 ('sandbox_35',),
 ('sandbox_36',),
 ('sandbox_37',),
 ('sandbox_38',),
 ('sandbox_39',),
 ('sandbox_4',),
 ('sandbox_40',),
 ('sandbox_41',),
 ('sandbox_42',),
 ('sandbox_43',),
 ('sandbox_44',),
 ('sandbox_45',),
 ('sandbox_46',),
 ('sandbox_47',),
 ('sandbox_48',),
 ('sandbox_49',),
 ('sandbox_5',),
 ('sandbox_50',),
 ('sandbox_51',),
 ('sandbox_52',),
 ('sandbox_53',),
 ('sandbox_5

In [65]:
# sanity check: each row contains a unique id in the column 'id'
con.execute("select count(id) as rows, count(distinct id) as ids from houses").fetchdf()

382909

In [18]:
con.execute("create table mappings as select id from houses").fetchall()

[(382909,)]

In [20]:
con.execute("select id from houses order by id").fetchdf()

,id
0,7003808130
1,7004010416
2,7004032234
3,7004041631
4,7004048100
...,...
382904,7051288544
382905,7051288700
382906,7051289458
382907,7051290755


In [89]:
con.execute("select * from mappings limit 10").fetchdf()

,id,sandbox_id
0,7050869455,1
1,7047358084,0
2,7041155838,0
3,7048757323,1
4,7044862988,0
5,7044756908,0
6,7043859584,0
7,7043677255,1
8,7043367923,1
9,7041521021,1


In [15]:
def split_table(tablename = "houses", n_chunks = None): # TODO: include parameter 'limit_rows'?
    N = con.execute("select count(id) as N from houses").fetchone()[0]
    print(N)

    # create mappings
    con.execute(f"""
    create or replace table mappings_sandbox as
    select id, mod(id, {n_chunks}) as sandbox_id from houses"""
    ).fetchdf()
    # split into 'n_chunks' tables
    for i in range(n_chunks):
        con.execute(f"""
        create or replace table sandbox_{i} as (
            select * from houses where mod(id, {n_chunks}) = {i}
        )
        """)

    # drop old tables
    drop_tables = [x[0] for x in con.execute("show tables").fetchall() if x[0][0:8]=="sandbox_" and int(x[0][8:])>=n_chunks]
    for t in drop_tables:
        con.execute(f"drop table {t}")

In [16]:
split_table(n_chunks = 100)
con.execute("select * from mappings limit 10").df()

382909


,id,sandbox_id
0,7050869455,55
1,7047358084,84
2,7041155838,38
3,7048757323,23
4,7044862988,88
5,7044756908,8
6,7043859584,84
7,7043677255,55
8,7043367923,23
9,7041521021,21


In [17]:
con.execute("show tables").fetchall()

[('dataset_test',),
 ('dataset_train',),
 ('houses',),
 ('mappings',),
 ('mappings_sandbox',),
 ('mappings_subset',),
 ('sandbox_0',),
 ('sandbox_1',),
 ('sandbox_10',),
 ('sandbox_11',),
 ('sandbox_12',),
 ('sandbox_13',),
 ('sandbox_14',),
 ('sandbox_15',),
 ('sandbox_16',),
 ('sandbox_17',),
 ('sandbox_18',),
 ('sandbox_19',),
 ('sandbox_2',),
 ('sandbox_20',),
 ('sandbox_21',),
 ('sandbox_22',),
 ('sandbox_23',),
 ('sandbox_24',),
 ('sandbox_25',),
 ('sandbox_26',),
 ('sandbox_27',),
 ('sandbox_28',),
 ('sandbox_29',),
 ('sandbox_3',),
 ('sandbox_30',),
 ('sandbox_31',),
 ('sandbox_32',),
 ('sandbox_33',),
 ('sandbox_34',),
 ('sandbox_35',),
 ('sandbox_36',),
 ('sandbox_37',),
 ('sandbox_38',),
 ('sandbox_39',),
 ('sandbox_4',),
 ('sandbox_40',),
 ('sandbox_41',),
 ('sandbox_42',),
 ('sandbox_43',),
 ('sandbox_44',),
 ('sandbox_45',),
 ('sandbox_46',),
 ('sandbox_47',),
 ('sandbox_48',),
 ('sandbox_49',),
 ('sandbox_5',),
 ('sandbox_50',),
 ('sandbox_51',),
 ('sandbox_52',),
 ('san

In [18]:
# creates table with columns: sandbox_id, sandbox_name, subset
def assign_subsets(subsets = {}):
    if len(subsets) == 0:
        print("error: no subsets have been specified")
        return 0
    
    # TODO: sanity check: the sum of subsets is equal to the total number of chunks
    n_chunks = len([x for x in con.execute("show tables").fetchall() if x[0][0:8]=="sandbox_"])
    if sum(subsets.values()) != n_chunks:
        print(f"error: the values need to sum to the number of analytical sandboxes ({n_chunks})")
        return 1

    con.execute("create or replace table mappings_subset (sandbox_id integer, sandbox_name string, subset string)")
    k=0
    for s in subsets:
        # TODO: if subsets[s] <= 0 : continue ???
        query_generate_table = f"create or replace view dataset_{s} as ( " # TODO: decide if view or table?
        query_generate_table_2 = []
        for i in range(subsets[s]):
            con.execute(
                f"insert into mappings_subset values (?,?,?)",
                [k, "sandbox_"+str(k), s]
                )
            query_generate_table_2.append("select * from sandbox_"+str(k))
            k += 1
        #create table subsets
        #print(query_generate_table + " union all ".join(query_generate_table_2) + ")")
        con.execute( query_generate_table + " union all ".join(query_generate_table_2) + ")")


    return 2



In [19]:
assign_subsets({"train":60, "validation":20, "test":20})

2

In [20]:
con.execute("select* from mappings_subset").df()

,sandbox_id,sandbox_name,subset
0,0,sandbox_0,train
1,1,sandbox_1,train
2,2,sandbox_2,train
3,3,sandbox_3,train
4,4,sandbox_4,train
...,...,...,...
95,95,sandbox_95,test
96,96,sandbox_96,test
97,97,sandbox_97,test
98,98,sandbox_98,test


In [4]:
con.close()